# Ordering

An `Ordering` specifies the order in which variables are eliminated during inference (e.g., Gaussian elimination, multifrontal QR). The choice of ordering significantly impacts the computational cost and fill-in (sparsity) of the resulting Bayes net or Bayes tree.

GTSAM provides several algorithms to compute good orderings automatically (like COLAMD, METIS) or allows you to specify a custom ordering.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/Ordering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [1]:
import gtsam
from gtsam import Ordering, VariableIndex
# Need a graph type for automatic ordering
from gtsam import SymbolicFactorGraph
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Creating an Ordering

Orderings can be created manually or computed automatically from a factor graph.

In [2]:
# Manual creation (list of keys)
manual_ordering = Ordering([X(1), L(1), X(2), L(2), X(0)])
manual_ordering.print("Manual Ordering: ")

Manual Ordering: Position 0: x1, l1, x2, l2, x0


In [ ]:
# Automatic creation requires a factor graph
# Let's use a simple SymbolicFactorGraph for structure
graph = SymbolicFactorGraph()
graph.push_factor(X(0))
graph.push_factor(X(0), X(1))
graph.push_factor(X(1), X(2))
graph.push_factor(X(0), L(1))
graph.push_factor(X(1), L(1))
graph.push_factor(X(1), L(2))
graph.push_factor(X(2), L(2))

# COLAMD (Column Approximate Minimum Degree) ordering
colamd_ordering = Ordering.Colamd(graph)
colamd_ordering.print("COLAMD Ordering: ")

# Constrained COLAMD (force x0 and x2 to be eliminated last)
constrained_ordering = Ordering.ColamdConstrainedLast(graph, gtsam.KeyVector([X(0), X(2)]))
constrained_ordering.print("Constrained COLAMD (x0, x2 last): ")

# METIS ordering (if GTSAM was built with METIS support)
try:
    metis_ordering = Ordering.Metis(graph)
    # metis_ordering.print("METIS Ordering: ")
except RuntimeError as e:
    print(f"\nSkipping METIS: {e}")

# Natural ordering (based on key magnitude)
natural_ordering = Ordering.Natural(graph)
# natural_ordering.print("Natural Ordering: ")

TypeError: Colamd(): incompatible function arguments. The following argument types are supported:
    1. (variableIndex: gtsam::VariableIndex) -> gtsam.gtsam.Ordering

Invoked with: kwargs: graph=SymbolicFactorGraph 
size: 7
factor 0:   x0
factor 1:   x0 x1
factor 2:   x1 x2
factor 3:   x0 l1
factor 4:   x1 l1
factor 5:   x1 l2
factor 6:   x2 l2


## Accessing Elements

An `Ordering` behaves like a vector of keys.

In [11]:
print(f"Ordering size: {colamd_ordering.size()}")

# Access by index
key_at_0 = colamd_ordering.at(0)
print(f"Key at position 0: {gtsam.DefaultKeyFormatter(key_at_0)}")

# Invert the ordering (map from Key to its position)
inverted = colamd_ordering.invert()
print("Inverted map (Key -> Position):")
for key, pos in inverted.items():
    print(f"  {gtsam.DefaultKeyFormatter(key)} -> {pos}")

Ordering size: 5
Key at position 0: l1


AttributeError: 'gtsam.gtsam.Ordering' object has no attribute 'invert'

## Appending Keys

You can append keys using `push_back`, `+=`, or `,`.

In [13]:
appended_ordering = Ordering(colamd_ordering)
appended_ordering.push_back(L(0))
appended_ordering.push_back(X(3))

appended_ordering.print("Appended Ordering: ")

Appended Ordering: Position 0: l1, x0, x1, l2, x2, l0, x3
